# 02 – Load Population Data

This notebook processes population data from Eurostat TSV file or downloads from API.

**What this step does:**
- **Primary**: Processes TSV file from `data/raw/population/demo_r_d2jan_tabular.tsv` (if available)
- **Fallback**: Downloads population data from Eurostat API (dataset `demo_r_pjangrp3` or alternatives)
- **Preserves all geographic levels**: Keeps country (2 chars), NUTS1 (3 chars), NUTS2 (4 chars), and NUTS3 (5+ chars) codes
- **Year range**: Processes data for 1990-2024 from TSV file, or downloads available years from API
- Caches the data to `data/processed/population_nuts2.parquet` for reuse

**Note**: All geographic levels are preserved for maximum flexibility in the star schema. Filtering to specific NUTS levels can be done during ETL or in queries.

**Shared utility**: This population data is used by both MVP (for curated dataset) and production (for database loading).


In [4]:
from pathlib import Path
import pandas as pd
from eurostat import get_data_df
import requests
import io

# Setup paths
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

PROJECT_ROOT


WindowsPath('C:/Users/narek.pirumyan/Desktop/IAE/2025/Big Data/Capstone Project/air-health-eu')

In [5]:
def _tidy_eurostat_population_tsv(path: Path, filter_nuts2: bool = False) -> pd.DataFrame:
    """
    Convert Eurostat population TSV into a tidy dataframe.
    Similar to _tidy_eurostat_tsv but specifically for population data.
    """
    df = pd.read_csv(path, sep="\t")
    dimension_col = df.columns[0]
    dimension_names = dimension_col.split(",")
    dims = df[dimension_col].str.split(",", expand=True)
    dims.columns = dimension_names

    wide_values = df.drop(columns=[dimension_col])
    tidy = pd.concat([dims, wide_values], axis=1)
    year_columns = [c for c in wide_values.columns if c.strip().replace(' ', '').isdigit()]

    tidy = tidy.melt(
        id_vars=dimension_names,
        value_vars=year_columns,
        var_name="year",
        value_name="population",
    )

    tidy["year"] = tidy["year"].astype(str).str.strip().astype(int)
    tidy["population"] = (
        tidy["population"]
        .astype(str)
        .str.strip()
        .replace(":", pd.NA)
    )
    tidy["population"] = pd.to_numeric(tidy["population"], errors="coerce")

    # Handle geo column (may be named 'geo' or 'geo\TIME_PERIOD')
    geo_col = None
    for col_name in tidy.columns:
        if 'geo' in col_name.lower() and 'time_period' in col_name.lower():
            geo_col = col_name
            break
    if not geo_col and 'geo' in tidy.columns:
        geo_col = 'geo'

    if geo_col:
        tidy["geo"] = tidy[geo_col].str.strip().str.upper()
        # Optional: Filter to NUTS2 level only (4 characters) if requested
        if filter_nuts2:
            def _is_valid_nuts2(code: str) -> bool:
                if not isinstance(code, str):
                    code = str(code)
                return len(code.strip()) == 4
            tidy = tidy[tidy["geo"].apply(_is_valid_nuts2)].copy()
        if geo_col != "geo" and geo_col in tidy.columns:
            tidy = tidy.drop(columns=[geo_col])
    
    # Filter to standard dimensions: freq=A, unit=NR, sex=T, age=TOTAL
    if 'freq' in tidy.columns:
        tidy = tidy[tidy['freq'] == 'A']
    if 'unit' in tidy.columns:
        tidy = tidy[tidy['unit'] == 'NR']
    if 'sex' in tidy.columns:
        tidy = tidy[tidy['sex'] == 'T']
    if 'age' in tidy.columns:
        tidy = tidy[tidy['age'] == 'TOTAL']
    
    # Keep only essential columns
    tidy = tidy[['geo', 'year', 'population']].dropna(subset=['population'])
    
    return tidy


def _process_population_tsv(tsv_path: Path, filter_nuts2: bool = False) -> pd.DataFrame:
    """
    Process population TSV file from data/raw/population.
    
    Parameters:
    - tsv_path: Path to the TSV file
    - filter_nuts2: If True, filter to NUTS2 level only
    
    Returns:
    - DataFrame with columns: geo, year, population
    """
    if not tsv_path.exists():
        raise FileNotFoundError(f"Population TSV file not found: {tsv_path}")
    
    print(f"  Processing TSV file: {tsv_path.name}")
    tidy = _tidy_eurostat_population_tsv(tsv_path, filter_nuts2=filter_nuts2)
    
    years = sorted(tidy['year'].unique())
    print(f"    ✓ Processed: years {min(years)}-{max(years)}, {len(tidy)} records")
    print(f"    ✓ Unique geographic codes: {tidy['geo'].nunique()}")
    
    return tidy


In [6]:
def _download_demo_r_d2jan() -> pd.DataFrame:
    """
    Download population data from demo_r_d2jan dataset using SDMX API.
    This dataset may have historical NUTS2 data going back to 2000.
    Tries both TSV and CSV formats.
    """
    import requests
    import io
    
    # Try CSV format first (easier to parse, includes labels)
    csv_url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/demo_r_d2jan/1.0"
    csv_params = {
        'compress': 'true',
        'format': 'csvdata',
        'formatVersion': '2.0',
        'lang': 'en',
        'labels': 'both'
    }
    
    try:
        print("  Downloading from demo_r_d2jan (SDMX API - CSV format)...")
        response = requests.get(csv_url, params=csv_params, timeout=120)
        response.raise_for_status()
        
        # Read CSV data
        df = pd.read_csv(io.StringIO(response.text), low_memory=False)
        
        # CSV format with labels should have columns like: freq, unit, sex, age, geo, TIME_PERIOD, OBS_VALUE
        # Or it might be in a different structure - let's check the columns
        print(f"    CSV columns: {list(df.columns)[:10]}...")  # Debug info
        
        # Look for common column patterns
        if 'TIME_PERIOD' in df.columns and 'OBS_VALUE' in df.columns:
            # Standard SDMX CSV format
            tidy = df[
                (df.get('freq', '') == 'A') &
                (df.get('unit', '') == 'NR') &
                (df.get('sex', '') == 'T') &
                (df.get('age', '') == 'TOTAL')
            ].copy()
            
            if not tidy.empty:
                tidy = tidy.rename(columns={'TIME_PERIOD': 'year', 'OBS_VALUE': 'population'})
                tidy['year'] = tidy['year'].astype(int)
                tidy['geo'] = tidy['geo'].str.strip().str.upper()
                tidy['population'] = pd.to_numeric(tidy['population'], errors='coerce')
                tidy = tidy.dropna(subset=['population'])
                tidy = tidy[(tidy['year'] >= 2000) & (tidy['year'] <= 2024)]
                return tidy[['geo', 'year', 'population']]
        
        # If CSV format doesn't work, try TSV format with filters
        print("    CSV format not as expected, trying TSV format with filters...")
        
    except Exception as e:
        print(f"    CSV format failed: {e}, trying TSV format...")
    
    # Fallback to TSV format with specific filters
    tsv_url = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/3.0/data/dataflow/ESTAT/demo_r_d2jan/1.0/*.*.*.*.*"
    tsv_params = {
        'c[freq]': 'A',
        'c[unit]': 'NR',
        'c[sex]': 'T',
        'c[age]': 'TOTAL',
        'c[geo]': 'EU27_2020,EU28,EU27_2007,BE,BE1,BE10,BE2,BE21,BE22,BE23,BE24,BE25,BE3,BE31,BE32,BE33,BE34,BE35,BG,BG3,BG31,BG32,BG33,BG34,BG4,BG41,BG42,CZ,CZ0,CZ01,CZ02,CZ03,CZ04,CZ05,CZ06,CZ07,CZ08,DK,DK0,DK01,DK02,DK03,DK04,DK05,DE,DE_TOT,DE1,DE11,DE12,DE13,DE14,DE2,DE21,DE22,DE23,DE24,DE25,DE26,DE27,DE3,DE30,DE4,DE40,DE5,DE50,DE6,DE60,DE7,DE71,DE72,DE73,DE8,DE80,DE9,DE91,DE92,DE93,DE94,DEA,DEA1,DEA2,DEA3,DEA4,DEA5,DEB,DEB1,DEB2,DEB3,DEC,DEC0,DED,DED2,DED4,DED5,DEE,DEE0,DEF,DEF0,DEG,DEG0,EE,EE0,EE00,IE,IE0,IE04,IE05,IE06,EL,EL3,EL30,EL4,EL41,EL42,EL43,EL5,EL51,EL52,EL53,EL54,EL6,EL61,EL62,EL63,EL64,EL65,ES,ES1,ES11,ES12,ES13,ES2,ES21,ES22,ES23,ES24,ES3,ES30,ES4,ES41,ES42,ES43,ES5,ES51,ES52,ES53,ES6,ES61,ES62,ES63,ES64,ES7,ES70,FR,FR1,FR10,FRB,FRB0,FRC,FRC1,FRC2,FRD,FRD1,FRD2,FRE,FRE1,FRE2,FRF,FRF1,FRF2,FRF3,FRG,FRG0,FRH,FRH0,FRI,FRI1,FRI2,FRI3,FRJ,FRJ1,FRJ2,FRK,FRK1,FRK2,FRL,FRL0,FRM,FRM0,FRY,FRY1,FRY2,FRY3,FRY4,FRY5,FRX,FRXX,HR,HR0,HR02,HR03,HR04,HR05,HR06,IT,ITC,ITC1,ITC2,ITC3,ITC4,ITF,ITF1,ITF2,ITF3,ITF4,ITF5,ITF6,ITG,ITG1,ITG2,ITH,ITH1,ITH2,ITH3,ITH4,ITH5,ITI,ITI1,ITI2,ITI3,ITI4,CY,CY0,CY00,LV,LV0,LV00,LT,LT0,LT01,LT02,LU,LU0,LU00,HU,HU1,HU11,HU12,HU2,HU21,HU22,HU23,HU3,HU31,HU32,HU33,HUX,HUXX,MT,MT0,MT00,NL,NL1,NL11,NL12,NL13,NL2,NL21,NL22,NL23,NL3,NL31,NL32,NL33,NL34,NL35,NL36,NL4,NL41,NL42,AT,AT1,AT11,AT12,AT13,AT2,AT21,AT22,AT3,AT31,AT32,AT33,AT34,PL,PL2,PL21,PL22,PL4,PL41,PL42,PL43,PL5,PL51,PL52,PL6,PL61,PL62,PL63,PL7,PL71,PL72,PL8,PL81,PL82,PL84,PL9,PL91,PL92,PT,PT1,PT11,PT15,PT16,PT17,PT18,PT19,PT1A,PT1B,PT1C,PT1D,PT2,PT20,PT3,PT30,RO,RO1,RO11,RO12,RO2,RO21,RO22,RO3,RO31,RO32,RO4,RO41,RO42,SI,SI0,SI03,SI04,SK,SK0,SK01,SK02,SK03,SK04,FI,FI1,FI19,FI1B,FI1C,FI1D,FI2,FI20,SE,SE1,SE11,SE12,SE2,SE21,SE22,SE23,SE3,SE31,SE32,SE33',
        'c[TIME_PERIOD]': '2024,2023,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991,1990',
        'compress': 'true',
        'format': 'tsv'
    }
    
    try:
        print("  Downloading from demo_r_d2jan (SDMX API - TSV format)...")
        response = requests.get(tsv_url, params=tsv_params, timeout=120)
        response.raise_for_status()
        
        # Read TSV data
        # The first column contains dimensions: freq,unit,sex,age,geo\TIME_PERIOD
        # Subsequent columns are years
        df = pd.read_csv(io.StringIO(response.text), sep='\t', low_memory=False, dtype=str)
        
        # Get the first column name (contains dimensions)
        first_col = df.columns[0]
        
        # Split first column into dimension columns
        # Format: freq,unit,sex,age,geo\TIME_PERIOD
        df[['freq', 'unit', 'sex', 'age', 'geo']] = df[first_col].str.split(',', expand=True, n=4)
        
        # Clean geo column (remove \TIME_PERIOD if present)
        df['geo'] = df['geo'].str.replace('\\TIME_PERIOD', '', regex=False)
        
        # Filter to required dimensions
        df = df[
            (df['freq'] == 'A') &
            (df['unit'] == 'NR') &
            (df['sex'] == 'T') &
            (df['age'] == 'TOTAL')
        ]
        
        # Get year columns (all columns except dimension columns)
        dim_cols = ['freq', 'unit', 'sex', 'age', 'geo', first_col]
        year_cols = [col for col in df.columns if col not in dim_cols]
        
        if not year_cols:
            raise ValueError("Could not find year columns in demo_r_d2jan data")
        
        # Melt to long format
        tidy = df.melt(
            id_vars=['geo'],
            value_vars=year_cols,
            var_name='year',
            value_name='population'
        )
        
        # Clean up
        tidy['year'] = tidy['year'].astype(int)
        tidy['geo'] = tidy['geo'].str.strip().str.upper()
        
        # Replace missing indicators (:, b, etc.) with NaN
        tidy['population'] = pd.to_numeric(tidy['population'], errors='coerce')
        tidy = tidy.dropna(subset=['population'])
        tidy['population'] = tidy['population'].astype(float)
        
        # Filter to desired year range
        tidy = tidy[(tidy['year'] >= 2000) & (tidy['year'] <= 2024)]
        
        return tidy[['geo', 'year', 'population']]
        
    except Exception as e:
        print(f"  Error downloading demo_r_d2jan (TSV): {e}")
        return pd.DataFrame()


def _try_alternative_datasets() -> dict:
    """
    Try multiple Eurostat population datasets to find historical data.
    Returns a dictionary with dataset availability information.
    """
    datasets = ['demo_r_d2jan', 'demo_r_pjangrp3', 'demo_r_pjangrp2', 'demo_r_pjangrp1']
    results = {}
    
    # Try demo_r_d2jan first (SDMX API)
    print("  Trying demo_r_d2jan (SDMX API)...")
    try:
        df_d2jan = _download_demo_r_d2jan()
        if not df_d2jan.empty:
            years = sorted(df_d2jan['year'].unique())
            nuts2_codes = df_d2jan[df_d2jan['geo'].str.len() == 4]['geo'].unique()
            results['demo_r_d2jan'] = {
                'available_years': (min(years), max(years)),
                'nuts2_count': len(nuts2_codes),
                'has_2000_2013': min(years) <= 2000 and max(years) >= 2013,
                'success': True,
                'dataframe': df_d2jan
            }
            print(f"    ✓ demo_r_d2jan: years {min(years)}-{max(years)}, {len(nuts2_codes)} NUTS2 regions")
        else:
            results['demo_r_d2jan'] = {'error': 'Empty dataframe', 'success': False}
    except Exception as e:
        results['demo_r_d2jan'] = {'error': str(e), 'success': False}
    
    # Try other datasets using eurostat package
    for dataset_code in ['demo_r_pjangrp3', 'demo_r_pjangrp2', 'demo_r_pjangrp1']:
        try:
            print(f"  Trying {dataset_code}...")
            df = get_data_df(dataset_code, flags=True)
            value_cols = [c for c in df.columns if c.endswith("_value")]
            if not value_cols:
                results[dataset_code] = {'error': 'No value columns found'}
                continue
                
            years = [int(c.replace("_value", "")) for c in value_cols]
            min_year, max_year = min(years), max(years)
            
            # Check NUTS2 availability
            if 'geo\\TIME_PERIOD' in df.columns:
                nuts2_codes = df[df['geo\\TIME_PERIOD'].str.len() == 4]['geo\\TIME_PERIOD'].unique()
                results[dataset_code] = {
                    'available_years': (min_year, max_year),
                    'nuts2_count': len(nuts2_codes),
                    'has_2000_2013': min_year <= 2000 and max_year >= 2013,
                    'success': True
                }
            else:
                results[dataset_code] = {'error': 'No geo column found'}
        except Exception as e:
            results[dataset_code] = {'error': str(e), 'success': False}
    
    return results


def _download_population(filter_nuts2: bool = False, try_alternatives: bool = True) -> pd.DataFrame:
    """
    Download population data from Eurostat and return tidy dataframe.
    
    Attempts to download data for years 2000-2024. Tries multiple datasets:
    1. demo_r_d2jan (SDMX API) - may have historical NUTS2 data
    2. demo_r_pjangrp3, demo_r_pjangrp2, demo_r_pjangrp1 (eurostat package)
    
    Note: Historical data availability varies by region and NUTS level. Some years
    (especially 2000-2013) may not be available at NUTS2 level for all regions.
    """
    # Try alternative datasets first if requested
    best_dataset = "demo_r_pjangrp3"
    best_dataframe = None
    
    if try_alternatives:
        print("Checking alternative Eurostat datasets for historical data...")
        alternatives = _try_alternative_datasets()
        
        # Find dataset with best coverage (has 2000-2013 and NUTS2 data)
        for dataset_code, info in alternatives.items():
            if info.get('success') and info.get('has_2000_2013') and info.get('nuts2_count', 0) > 0:
                best_dataset = dataset_code
                # If demo_r_d2jan, use the pre-downloaded dataframe
                if dataset_code == 'demo_r_d2jan' and 'dataframe' in info:
                    best_dataframe = info['dataframe']
                print(f"  ✓ Found {dataset_code} with years {info['available_years']} and {info['nuts2_count']} NUTS2 regions")
                break
        else:
            print(f"  ⚠ No alternative dataset found with 2000-2013 NUTS2 data, using {best_dataset}")
    
    # If we have a pre-downloaded dataframe (from demo_r_d2jan), use it
    if best_dataframe is not None:
        tidy = best_dataframe.copy()
    else:
        # Download from best available dataset using eurostat package
        df = get_data_df(best_dataset, flags=True)
        value_cols = [c for c in df.columns if c.endswith("_value")]
        tidy = df.melt(
            id_vars=["freq", "sex", "unit", "age", "geo\\TIME_PERIOD"],
            value_vars=value_cols,
            var_name="year_raw",
            value_name="population",
        )
        tidy["year"] = tidy["year_raw"].str.replace("_value", "", regex=False).astype(int)
        tidy = tidy.rename(columns={"geo\\TIME_PERIOD": "geo"})
        tidy = tidy[
            (tidy["freq"] == "A")
            & (tidy["sex"] == "T")
            & (tidy["unit"] == "NR")
            & (tidy["age"] == "TOTAL")
        ]
        tidy = tidy[["geo", "year", "population"]]
        tidy["geo"] = tidy["geo"].str.strip().str.upper()
        tidy = tidy.dropna(subset=["population"])
        tidy["population"] = tidy["population"].astype(float)
        
        # Filter to desired year range (2000-2024) if available
        tidy = tidy[(tidy["year"] >= 2000) & (tidy["year"] <= 2024)]
    
    # Optional: Filter to NUTS2 level only (4 characters) if requested
    # By default, we preserve all geographic levels for maximum flexibility
    if filter_nuts2:
        def _is_valid_nuts2(code: str) -> bool:
            if not isinstance(code, str):
                code = str(code)
            return len(code.strip()) == 4
        tidy = tidy[tidy["geo"].apply(_is_valid_nuts2)]
    
    return tidy

def load_population(force_refresh: bool = False, filter_nuts2: bool = False, try_alternatives: bool = True) -> pd.DataFrame:
    """
    Load population data, processing TSV file if available, otherwise downloading from API.
    
    Parameters:
    - force_refresh: If True, re-process/download even if cached file exists
    - filter_nuts2: If True, filter to NUTS2 level only (4-character codes)
    - try_alternatives: If True, try alternative Eurostat datasets for historical data (only if TSV not available)
    """
    population_path = PROJECT_ROOT / "data" / "processed" / "population_nuts2.parquet"
    tsv_path = PROJECT_ROOT / "data" / "raw" / "population" / "demo_r_d2jan_tabular.tsv"
    
    # Check if cached parquet exists and we're not forcing refresh
    if population_path.exists() and not force_refresh:
        df = pd.read_parquet(population_path)
        # Apply filtering if requested and data exists
        if filter_nuts2:
            def _is_valid_nuts2(code: str) -> bool:
                if not isinstance(code, str):
                    code = str(code)
                return len(code.strip()) == 4
            df = df[df["geo"].apply(_is_valid_nuts2)]
        return df

    # Try to process TSV file first (if available)
    if tsv_path.exists():
        print("Found TSV file, processing...")
        try:
            tidy = _process_population_tsv(tsv_path, filter_nuts2=filter_nuts2)
            population_path.parent.mkdir(parents=True, exist_ok=True)
            tidy.to_parquet(population_path, index=False)
            return tidy
        except Exception as e:
            print(f"  ⚠ Error processing TSV file: {e}")
            print("  Falling back to API download...")
    else:
        print("TSV file not found, downloading from API...")
    
    # Fallback to API download
    tidy = _download_population(filter_nuts2=filter_nuts2, try_alternatives=try_alternatives)
    population_path.parent.mkdir(parents=True, exist_ok=True)
    tidy.to_parquet(population_path, index=False)
    return tidy

# Load population data (preserving all geographic levels)
# Set try_alternatives=True to check for historical data in other Eurostat datasets
population = load_population(force_refresh=False, filter_nuts2=False, try_alternatives=True)
print(f"✓ Loaded {len(population):,} population records")
print(f"  Years: {population['year'].min()} - {population['year'].max()}")
print(f"  Unique geographic codes: {population['geo'].nunique()}")
print(f"  Geographic code length distribution:")
print(f"    {population['geo'].str.len().value_counts().sort_index().to_dict()}")

# Check NUTS2 coverage
nuts2_data = population[population['geo'].str.len() == 4]
if len(nuts2_data) > 0:
    nuts2_years = sorted(nuts2_data['year'].unique())
    print(f"\n  NUTS2 data available for years: {nuts2_years[0]} - {nuts2_years[-1]} ({len(nuts2_years)} years)")
    if nuts2_years[0] > 2000:
        print(f"  ⚠ WARNING: NUTS2 data starts from {nuts2_years[0]}, not 2000")
        print(f"     Consider using NUTS1 interpolation or accepting NULL values for earlier years")

population.head()


✓ Loaded 14,830 population records
  Years: 1990 - 2024
  Unique geographic codes: 521
  Geographic code length distribution:
    {2: 1228, 3: 3793, 4: 9768, 6: 15, 9: 26}

  NUTS2 data available for years: 1990 - 2024 (35 years)


,geo,year,population
0,AL,1990,3286500.0
1,AL0,1990,3286500.0
2,AT,1990,7644818.0
3,AT1,1990,3219274.0
4,AT11,1990,270670.0
